In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly 

In [3]:
# Download Life expectancy based on economic freedom with time line

'''
Life expectancy:
https://www.kaggle.com/kumarajarshi/life-expectancy-who

Economic freedom 
https://www.kaggle.com/gsutters/economic-freedom 
'''

'\nLife expectancy:\nhttps://www.kaggle.com/kumarajarshi/life-expectancy-who\n\nEconomic freedom \nhttps://www.kaggle.com/gsutters/economic-freedom \n'

In [4]:
# File location 

economic_freedom = 'drive/My Drive/NYU/Programming_for_DS/Data/efw_cc.csv'
life_expectancy  = 'drive/My Drive/NYU/Programming_for_DS/Data/Life Expectancy Data.csv'

In [5]:
df_ef = pd.read_csv(economic_freedom)
df_le = pd.read_csv(life_expectancy)

In [6]:
#Common years between two datasets

ef_date = df_ef.year.unique()
df_date = df_le.Year.unique()
common_years = np.intersect1d(ef_date, df_date)

print(common_years)

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015]


In [7]:
# Economic Freedom

df_ef.head()

,year,ISO_code,countries,ECONOMIC FREEDOM,rank,quartile,1a_government_consumption,1b_transfers,1c_gov_enterprises,1d_top_marg_tax_rate,1_size_government,2a_judicial_independence,2b_impartial_courts,2c_protection_property_rights,2d_military_interference,2e_integrity_legal_system,2f_legal_enforcement_contracts,2g_restrictions_sale_real_property,2h_reliability_police,2i_business_costs_crime,2j_gender_adjustment,2_property_rights,3a_money_growth,3b_std_inflation,3c_inflation,3d_freedom_own_foreign_currency,3_sound_money,4a_tariffs,4b_regulatory_trade_barriers,4c_black_market,4d_control_movement_capital_ppl,4_trade,5a_credit_market_reg,5b_labor_market_reg,5c_business_reg,5_regulation
0,2016,ALB,Albania,7.54,34.0,1.0,8.232353,7.509902,8.0,8.0,7.935564,2.668222,3.145462,4.512228,8.333333,4.166667,4.387444,6.485287,6.933500,6.215401,0.948718,5.071814,8.986454,9.484575,9.74360,10.0,9.553657,8.963556,7.489905,10.00000,6.406138,8.214900,7.098562,6.916278,6.705863,6.906901
1,2016,DZA,Algeria,4.99,159.0,4.0,2.150000,7.817129,0.0,4.5,3.616782,4.186704,4.327113,4.689952,4.166667,5.000000,4.507538,6.626692,6.136845,6.737383,0.820513,4.690743,6.955962,8.339152,8.72046,5.0,7.253894,6.872533,2.481294,5.56391,1.590362,4.127025,5.100509,5.029513,5.676956,5.268992
2,2016,AGO,Angola,5.17,155.0,4.0,7.600000,8.886739,0.0,9.5,6.496685,1.843129,1.974566,2.512364,3.333333,4.166667,2.302200,5.455882,3.016104,4.291197,0.846154,2.963635,9.385679,4.986742,3.05400,5.0,5.606605,6.989244,2.024949,10.00000,2.044823,5.264754,7.064905,4.560325,4.930271,5.518500
3,2016,ARG,Argentina,4.84,160.0,4.0,5.335294,6.048930,6.0,4.0,5.346056,3.689690,2.930563,4.255995,7.500000,3.333333,3.632872,6.857195,3.385786,4.133832,0.769231,3.904030,5.233264,5.224079,2.00000,10.0,5.614336,6.421600,4.811105,0.00000,4.697482,3.982547,5.419820,5.151405,5.535831,5.369019
4,2016,ARM,Armenia,7.57,29.0,1.0,7.264706,7.748532,8.0,5.0,7.003310,3.867379,4.197569,5.664317,5.833333,5.000000,5.196693,9.801963,5.714028,7.007654,1.000000,5.809215,9.078264,9.263496,9.74600,10.0,9.521940,8.547556,7.194410,10.00000,6.830998,8.143241,9.102046,6.234630,6.797530,7.378069


In [8]:
# Life Expectancy

df_le.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [9]:
#Renaming columns to make year and county column name consistant b/w df's

df_ef['Year']      = df_ef['year']
df_ef['Country']   = df_ef['countries']
df_le['Life expectancy'] = df_le['Life expectancy ']

In [10]:
# Merging dataframe

merged_df = df_ef.merge(df_le, on=['Year', 'Country'])

In [11]:
# Interested in Life expectancy and ECONOMIC FREEDOM

interested_cols = ['Year', 'ISO_code','Country','ECONOMIC FREEDOM', 'Adult Mortality', 'infant deaths', 'Life expectancy']
clean_df = merged_df[interested_cols]

In [12]:
# Check if : (all the merged rows)  and (where count is ==16 rows) are same

print(clean_df.groupby(by='Country').count()['Year'].shape[0])
print(clean_df.groupby(by='Country').count() [ clean_df.groupby(by='Country').count()['Year']==16].shape[0])

133
133


In [13]:
# Checking Unique Years in clean_df

clean_df.Year.unique()

array([2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005,
       2004, 2003, 2002, 2001, 2000])

In [14]:
# Plotting time series 

clean_df = clean_df.dropna()
data_slider = []
steps = []

# Popluate data_slider
for year in clean_df.Year.unique():
    if year !=2000:
      df1   = clean_df[clean_df['Year']== year]
      df2 = clean_df[clean_df['Year']== 2000]
      # Dict with the data for the current year
      
      temp_Df = df1.merge(df2, on=['ISO_code'])
      temp_Df['delta_ef'] = temp_Df["ECONOMIC FREEDOM_x"] - temp_Df["ECONOMIC FREEDOM_y"]
      temp_Df['delta_lf'] = temp_Df["Life expectancy_x"]	- temp_Df["Life expectancy_y"]
      temp_Df.dropna()

      # Column z: Higher value means more correlation 
      temp_Df['z'] = abs(temp_Df['delta_ef']*temp_Df['delta_lf'] )

      data_one_year = dict(
                          type='choropleth',
                          locations = temp_Df['ISO_code'],
                          z=temp_Df['z'],
                          locationmode='ISO-3',
                          colorscale = 'Sunsetdark'
                          )

      data_slider.append(data_one_year) 

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                #Label to be displayed for each step (year)
                label='Year {}'.format(i + 2001)) 
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  
layout = dict(geo=dict(scope='world'), sliders=sliders)
fig = dict(data=data_slider, layout=layout) 

plotly.offline.iplot(fig)